In [24]:
%matplotlib inline
%run spherical.py

from IPython.display import display
import sympy
from sympy import *
from sympy.parsing.sympy_parser import *
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import functools as ft
import random

init_printing(use_latex='mathjax')

theta_r, theta_o, theta_i = symbols("\\theta_{range} \\theta_o \\theta_i")
theta_min, theta_max = symbols("\\theta_min \\theta_max")
theta_spec = pi - theta_o

# Neulander Shading Function Normalization

## Notations

 - $t$: Unit tangent vector
 - $\omega_o$: Unit outgoing vector
 - $\theta_o$: Angle between $t$ and $\omega_o$.
 - $\omega_o^\perp$: Projection of $\omega_o$ on the plane perpendicular to $t$:
$$\omega_o^\perp = \frac{\omega_o - (t \cdot \omega_o)t}{||\omega_o - (t \cdot \omega_o)t ||}$$
 - $b = \omega_o \times t$
 - $n(\theta) = b.\cos \theta + \omega_o'.\sin \theta$: parametrization of normal vectors of the hair cylinder
 - $\omega_i$: Unit incident vector
 - $\theta_i$: Angle between $t$ and $\omega_i$.
 - $\omega_{spec}$: Perfect specular reflection of $\omega_o$ around the normal contained in both planes $(\omega_o, \omega_i)$ and $(b, \omega_o^\perp)$
 - $\theta_{spec} = \pi - \theta_o$: Angle between $t$ and $\omega_{spec}$.

## Shading model

$$
\begin{align}
\Psi_{neulander}(\omega_o,\omega_i) 
&=
\left\{
\begin{split}
&k_s \max(0, \cos (\theta_{spec} - \theta_i)) \text{ if } \theta_i \in [\theta_{min}, \theta_{max}]\\
&0 \text{ otherwise }
\end{split}
\right. \\
&=
\left\{
\begin{split}
&k_s \max(0, \cos \theta_{spec} \cos \theta_i + \sin \theta_{spec} \sin \theta_i ) \text{ if } \theta_i \in [\theta_{min}, \theta_{max}]\\
&0 \text{ otherwise }
\end{split}
\right. \\
&=
\left\{
\begin{split}
&k_s \max(0, -\cos \theta_i \cos \theta_o + \sin \theta_i \sin \theta_o) \text{ if } \theta_i \in [\theta_{min}, \theta_{max}]\\
&0 \text{ otherwise }
\end{split}
\right.
\end{align}
$$

where $\theta_{min}, \theta_{max}$ are parameters to control the solid angle where rays scatter.

## Normalization

Given $\omega_o \in \Omega_{4\pi}$, we search $c_o = \int_{\omega_i \in \Omega_{4\pi}} (-\cos \theta_o \cos \theta_i + \sin \theta_o \sin \theta_i)d\sigma(\omega_i)$.

$$
\begin{align}
c_o &=
\int_{\phi_i = 0}^{2\pi}\int_{\theta_i=\theta_{min}}^{\theta_{max}} (-\cos \theta_o \cos \theta_i + \sin \theta_o \sin \theta_i) \sin \theta_i d\theta_i d\phi_i \\
&=
2\pi (-\cos \theta_o\int_{\theta_i=\theta_{min}}^{\theta_{max}} \cos \theta_i \sin \theta_i d\theta_i + \sin\theta_o \int_{\theta_i=\theta_{min}}^{\theta_{max}} \sin^2 \theta_i d\theta_i) \\
&=
2\pi (-I_1\cos \theta_o + I_2\sin\theta_o)
\end{align}
$$

$I_1$ and $I_2$ are computed by the following code:

In [20]:
I_1 = integrate(cos(theta_i) * sin(theta_i), (theta_i, theta_min, theta_max))
I_2 = integrate(sin(theta_i) * sin(theta_i), (theta_i, theta_min, theta_max))

In [4]:
display(I_1)

   2                  2            
sin (\theta_max)   sin (\theta_min)
──────────────── - ────────────────
       2                  2        

In [5]:
display(I_2)

\theta_max   \theta_min   sin(\theta_max)⋅cos(\theta_max)   sin(\theta_min)⋅co
────────── - ────────── - ─────────────────────────────── + ──────────────────
    2            2                       2                                 2  

s(\theta_min)
─────────────
             

In [6]:
c_o = simplify(2*pi*(-cos(theta_o) * I_1 + sin(theta_o) * I_2))
display(c_o)

  ⎛                                                    cos(2⋅\theta_max + \the
π⋅⎜\theta_max⋅sin(\thetaₒ) - \theta_min⋅sin(\thetaₒ) + ───────────────────────
  ⎝                                                                 2         

taₒ)   cos(2⋅\theta_min + \thetaₒ)⎞
──── - ───────────────────────────⎟
                    2             ⎠

An interesting way of choosing parameters $\theta_{min}$ and $\theta_{max}$ is to fix a range $\theta_{range}$ and to set $\theta_{min} = \theta_{spec} - \theta_{range}$ and $\theta_{max} = \theta_{spec} + \theta_{range}$.
In that case we have:

In [9]:
theta_min = theta_spec - theta_r
theta_max = theta_spec + theta_r

I_1 = integrate(cos(theta_i) * sin(theta_i), (theta_i, theta_min, theta_max))
I_2 = integrate(sin(theta_i) * sin(theta_i), (theta_i, theta_min, theta_max))

In [10]:
display(I_1)

   2                                2                          
sin (\thetaₒ - \theta_{range})   sin (\thetaₒ + \theta_{range})
────────────────────────────── - ──────────────────────────────
              2                                2               

In [11]:
display(I_2)

                 sin(\thetaₒ - \theta_{range})⋅cos(\thetaₒ - \theta_{range})  
\theta_{range} + ─────────────────────────────────────────────────────────── -
                                              2                               

 sin(\thetaₒ + \theta_{range})⋅cos(\thetaₒ + \theta_{range})
 ───────────────────────────────────────────────────────────
                              2                             

In [12]:
c_o = simplify(2*pi*(-cos(theta_o) * I_1 + sin(theta_o) * I_2))
display(c_o)

2⋅π⋅(\theta_{range} + sin(\theta_{range})⋅cos(\theta_{range}))⋅sin(\thetaₒ)

Let's now test experimentally our formula for the normalization factor by estimating $\frac{1}{c_o} \int_{\omega_i \in \Omega_{4\pi}} \Psi_{neulander}(\omega_o,\omega_i) d\omega_i$ with Monte Carlo integration. It should converge to 1 for each $\omega_o$ and possible range $\theta_{range}$.

In [ ]:
#plt.xlim(0, float(pi))

#theta_range_ = np.pi / 2
#xs = np.linspace(0, float(pi), 256)
#plt.plot(xs, np.array([float(c_o.subs(theta_o, x).subs(theta_r, theta_range_)) for x in xs]))
#plt.show()

# Evaluate \Psy(w_o, w_i) / c_o
def neulander_eval(theta_range, w_o, w_i):
    global theta_r
    global theta_o
    global c_o
    
    cos_o = w_o[2];
    cos_i = w_i[2];

    theta_o_value = np.arccos(cos_o)
    theta_spec = np.pi - theta_o_value
    theta_min = max(0, theta_spec - theta_range)
    theta_max = min(np.pi, theta_spec + theta_range)
    cos_theta_min = np.cos(theta_min)
    cos_theta_max = np.cos(theta_max)
    
    if cos_i > cos_theta_min or cos_i < cos_theta_max:
        return 0
    
    sin_i = cos2sin(cos_i)
    sin_o = cos2sin(cos_o)
    
    c = float(c_o.subs(theta_o, theta_o_value).subs(theta_r, theta_range))
    
    return max(0, -cos_i * cos_o + sin_i * sin_o) / c

random.seed(0)

size = 256 # Estimation with 256 directions
randu = np.array([random.random() for i in range(size)])
randv = np.array([random.random() for i in range(size)])

dirs = [sample_uniform_sphere(randu[i], randv[i]) for i in range(len(randu))]

print("test neulander normalization")

mean_estimates = 0

rs = np.linspace(0.01, float(pi) / 2, 32)
for r in rs:
    current_estimate = 0
    for w_o in dirs:
        estimate = (1 / size) * sum((neulander_eval(r, w_o, w_i) / pdf_uniform_sphere(w_i) for w_i in dirs))
        current_estimate = current_estimate + estimate
    current_estimate = current_estimate / len(dirs) # Average estimates
    print("estimate for r = ", r, " is ", current_estimate)
    mean_estimates = mean_estimates + current_estimate

mean_estimates = mean_estimates / len(rs)
print("complete mean estimate = ", mean_estimates)